In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect("sites_database_promo.db") # или :memory: чтобы сохранить в RAM
cursor = conn.cursor()

In [3]:
cursor.execute("""CREATE TABLE site
                  (Id INTEGER PRIMARY KEY,title TEXT NOT NULL)
               """)
cursor.execute("""CREATE TABLE classes
                  (Id INTEGER PRIMARY KEY,title TEXT NOT NULL)
               """)
cursor.execute("""CREATE TABLE site_class (
 
site_id INTEGER NOT NULL,
 
class_id INTEGER NOT NULL,
 
FOREIGN KEY (site_id) REFERENCES site(id)
 
FOREIGN KEY (class_id) REFERENCES class(id)
 
)
               """)

In [4]:
import zipfile as z
import os

In [5]:
data = z.ZipFile(os.getcwd() + '\zproxy_db2.zip')

In [6]:
domains_names = [line for line in data.namelist() if 'domains' in line]
#os.mkdir(os.getcwd() + '\domains')
for file in domains_names:
    data.extract(file, path=os.getcwd() + '\domains')

In [21]:
import re
from tqdm import tqdm_notebook
pattern = '(?<=sites\/)[A-Za-z0-9., А-Яа-я-]*(?=\/domains)'

sites = {}
class_id=1
site_id=0
insert_class = "INSERT INTO classes VALUES (?,?)"
insert_site = "INSERT INTO site VALUES (?,?)"
insert_site_class = "INSERT INTO site_class VALUES (?,?)"

for site in tqdm_notebook(domains_names):
    name = re.findall(pattern, site)[0]
    cursor.execute(insert_class, (class_id,name))
    with open('C:/Kate/web-crawler/zproxy_db2/' + site) as file:
        sites[name] = file.read().split('\n')
        sites[name].remove('')
        if len(sites[name])>10000:
            sites[name]=sites[name][:10000]
        for sit in sites[name]:
            cursor.execute("SELECT EXISTS(SELECT 1 FROM site WHERE title=?)",(sit,))
            a=cursor.fetchall()[0][0]
            if (a!=1):
                site_id=site_id+1
                cursor.execute(insert_site, (site_id,sit)) 
                cursor.execute(insert_site_class, (site_id,class_id))
            else:
                cursor.execute("SELECT site.Id FROM site WHERE site.title=?",(sit,))
                a=cursor.fetchall()[0][0]
                cursor.execute(insert_site_class, (a,class_id))
                
            
    class_id=class_id+1

In [22]:
conn.commit()

In [20]:
sql = "DELETE FROM site_class"
cursor.execute(sql)
conn.commit()
sql = "DELETE FROM site"
cursor.execute(sql)
conn.commit()
sql = "DELETE FROM classes"
cursor.execute(sql)
conn.commit()

In [23]:
sql = "SELECT * FROM classes"
cursor.execute(sql)
print(cursor.fetchall())

[(1, 'Авто,  Мото,  Транспорт'), (2, 'Азартные игры'), (3, 'Алкоголь,  Табакокурение'), (4, 'Анимация'), (5, 'Анонимные прокси,  TOR,  VPN-прокси'), (6, 'Армия,  Оружие'), (7, 'Архивы программ'), (8, 'Баннеры,  Реклама'), (9, 'Бесплатный хостинг'), (10, 'Биткоин'), (11, 'Благотворительность'), (12, 'Веб порталы'), (13, 'Веб радио'), (14, 'Веб телевидение'), (15, 'Веб телефон'), (16, 'Веб-почта'), (17, 'Вредоносное ПО,  Вирусы'), (18, 'Государство,  Политика и право'), (19, 'Дети,  Семья,  Школа'), (20, 'Дизайн,  Архитектура'), (21, 'Дом,  Дача,  Ремонт'), (22, 'Домашние животные'), (23, 'Доски объявлений'), (24, 'Запаркованные домены'), (25, 'Заработок в Интернет'), (26, 'Игры'), (27, 'Компьютеры,  Интернет'), (28, 'Криптовалюта'), (29, 'Кулинария,  Еда,  Напитки'), (30, 'Культура,  Искусство,  Классика'), (31, 'Литература,  Тексты произведений'), (32, 'Медицина,  Здоровье'), (33, 'Мода и красота'), (34, 'Музыка,  Видео,  Изображения,  Торренты'), (35, 'Наркотики'), (36, 'Насилие,  Рас